In [1]:
from datasets import load_dataset
from random import randrange

import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer

from huggingface_hub import login

import wandb

2023-09-21 00:11:31.011621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 00:11:32.791839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

wandb.login()
%env WANDB_PROJECT=python-fine-tuning

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: timkh0625 (seclab_kiho). Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=python-fine-tuning


In [3]:
model_name = "Salesforce/codegen25-7b-mono"
dataset_name = "/data/kiho/autocode/codegen/finetuning/llama-peft-tuner/code_segments_py150k_sanitized_20"
# device_map = {"": 0}
device_map = "auto"
split = "train[:10%]"
finetunes_model_name = "coodegen25-7b-finetuned-int4-python150k"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)


In [4]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

    ### Response:
    {sample['text']}
    """

In [5]:
dataset = load_dataset(dataset_name, split=split)

base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, use_cache = False, device_map=device_map, trust_remote_code=True, cache_dir='/data/kiho/autocode/codegen')
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [8]:
# Define the training arguments
trainingArgs = TrainingArguments(
    output_dir=finetunes_model_name, # Output directory where the model predictions and checkpoints will be stored
    num_train_epochs=3, # Number of training epochs
    per_device_train_batch_size=32, # Batch size per GPU for training
    gradient_accumulation_steps=4,  # Number of update steps to accumulate the gradients for
    gradient_checkpointing=True,# Enable gradient checkpointing
    optim="paged_adamw_32bit", # Optimizer to use
    save_steps=250,
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    #tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    max_steps=500,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    report_to="wandb",
    seed=42
)
# Create the trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=512,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)


In [9]:
print("Start the supervised Fine tuning")
trainer.train()
print("Done Training")

#stop reporting to wandb
wandb.finish()

# save model
trainer.save_model()
print("Model saved")

Start the supervised Fine tuning


Token indices sequence length is longer than the specified maximum sequence length for this model (2199 > 2048). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [14]:
import gc

# clear the VRAM
# del base_model
# del trained_model
# del lora_merged_model
del trainer
torch.cuda.empty_cache()
gc.collect()

19409